# Laboratorio 5: Análisis de Red

## Parte 1: Análisis de Paquetes

### **Análisis Estadístico**

In [69]:
# Imports
from scapy.all import *
import pandas as pd
import matplotlib.pyplot as plt


In [72]:
# Lectura de paquetes recurrentes y unificación de .pcap preexistente 
conf.promisc = True
# Cargar el archivo pcap existente
paquetes_pcap = rdpcap('analisis_paquetes.pcap')
payload = paquetes_pcap[0].load
# Capturar 10 paquetes en la interfaz de red "Wi-Fi"
conf.promisc = True
paquetes_capturados = sniff(count=10, iface="Wi-Fi")

# Combinar los paquetes capturados y los del archivo pcap en una lista
paquetes_total = PacketList(paquetes_capturados + paquetes_pcap)

# Crear el DataFrame con las columnas solicitadas
df = pd.DataFrame({'Src Address': [p[IP].src for p in paquetes_total if IP in p],
                   'Dst Address': [p[IP].dst for p in paquetes_total if IP in p],
                   'Src Port': [p[TCP].sport if TCP in p else p[UDP].sport for p in paquetes_total if IP in p],
                   'Dst Port': [p[TCP].dport if TCP in p else p[UDP].dport for p in paquetes_total if IP in p],
                   'Payload': [int.from_bytes(p[Raw].load, byteorder='big') if Raw in p else 0 for p in paquetes_total if IP in p]})


In [73]:
df.sample(n=5)
df['Payload'].loc[42]

0

### Estadísticas

In [14]:
# IP Origen más frecuente

ip_mas_frecuente = df['Src Address'].mode()[0]
print("La dirección IP de origen más frecuente es:", ip_mas_frecuente)

La dirección IP de origen más frecuente es: 10.1.10.53


In [15]:
# IP Destino más frecuente

ip_mas_frecuente = df['Dst Address'].mode()[0]
print("La dirección IP de origen más frecuente es:", ip_mas_frecuente)

La dirección IP de origen más frecuente es: 10.1.10.53


In [16]:
# ¿A qué IPs se comunica la IP de Origen más frecuente?

ip_origen_mas_frecuente = df['Src Address'].mode()[0]
ips_destino = df.loc[df['Src Address'] == ip_origen_mas_frecuente, 'Dst Address'].unique()
print("La dirección IP de origen más frecuente es:", ip_origen_mas_frecuente)
print("Se comunica con las siguientes direcciones IP de destino:", ips_destino)

La dirección IP de origen más frecuente es: 10.1.10.53
Se comunica con las siguientes direcciones IP de destino: ['84.54.22.33' '75.75.75.75']


In [20]:
# ¿A qué puertos destino se comunica la IP de Origen más frecuente?

ip_mas_frecuente = df['Src Address'].value_counts().index[0]
df_ip_frecuente = df[df['Src Address'] == ip_mas_frecuente]
puertos_destino_unicos = df_ip_frecuente['Dst Port'].unique()
print('Se comunica con los siguientes puertos',puertos_destino_unicos)

Se comunica con los siguientes puertos [53]


In [22]:
# ¿A qué puertos origen se comunica la IP de Destino más frecuente?

ip_mas_frecuente = df['Dst Address'].value_counts().index[0]
df_ip_frecuente = df[df['Dst Address'] == ip_mas_frecuente]
puertos_origen_unicos = df_ip_frecuente['Src Port'].unique()
print('Se comunica con los siguientes puertos',puertos_origen_unicos)

Se comunica con los siguientes puertos [53]


### Indique el propósito de los puertos que más aparece en los incisos anteriores
<br>
<div style="text-align: justify">
    El puerto <b>53</b> se utiliza comúnmente para el protocolo de sistema de nombres de dominio (DNS), que permite la resolución de nombres de dominio en direcciones IP. Cuando un usuario ingresa una URL en un navegador web, el navegador envía una solicitud DNS al servidor DNS designado para obtener la dirección IP correspondiente del servidor web que aloja la página web solicitada. Esta solicitud se realiza en el puerto 53 del servidor DNS. Por lo tanto, el puerto 53 es esencial para la navegación web y para la comunicación en línea en general.
</div>